In [1]:
import altair as alt
import pandas as pd
from altair import datum

from pyprojroot import here

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

df = get_processed_dataset("austria_country")

In [2]:
country = df[df.BundeslandID == 10].copy()
country['New_Tests'] = country.TestGesamt.diff().iloc[50:]
country["Positivity_Rate"] = (100.0 * country.AnzahlFaelle / country.New_Tests).iloc[50:]
country["Cumulative_Positivity_Rate"] = (100.0 * country.AnzahlFaelleSum / country.TestGesamt).iloc[50:]

In [3]:
today = country.tail(1)
tests = today.New_Tests.values[0]
tabella = {"Data": str(today.Date.values[0]), 
           "Nuovi casi": int(today.AnzahlFaelle), 
           "Nuovi deceduti": int(today.AnzahlTotTaeglich), 
           "Tests": tests, 
           "Incidenza media 7 giorni": float(today.Incidence)}

tabella

{'Data': '2020-11-16T00:00:00.000000000',
 'Nuovi casi': 5100,
 'Nuovi deceduti': 40,
 'Tests': 26232.0,
 'Incidenza media 7 giorni': 747.0214285714285}

In [4]:
resize = alt.selection_interval(bind='scales')

def rolling_avg(
    field,
    title,
    data=country,
    tooltip=["Date", "AnzahlFaelle", "RollingAvg"],
):
    base = (
        alt.Chart()
        .transform_window(averaged=f"mean({field})", frame=[-4, 3])
        .encode(x="Date:T", tooltip=tooltip)
        .properties(height=200, width=600,)
    )

    c1 = base.mark_line().encode(y=alt.Y("averaged:Q", title=""),)

    c2 = base.mark_bar(opacity=0.1).encode(
        y=alt.Y(f"{field}:Q", title=""),
    )

    return (
        alt.layer(c1, c2, data=data)
        .properties(title={"text": [title], "subtitle": ["Dato mediato su 7 giorni"]})
        .add_selection(
            resize
        )
    )

In [11]:
c5 = alt.Chart(country).mark_line().encode(x="Date:T", y=alt.Y("Incidence", title='')).properties(height=200, width=600,title='Incidenza media')
c6 = alt.Chart(country).mark_line().encode(x="Date:T", y=alt.Y("FZICU", title='')).properties(height=200, width=600,title='Occupazione terapia intensiva')

c1 = rolling_avg('AnzahlFaelle', title='Nuovi casi positivi', data=country)
#c5 = rolling_avg('Incidence', title='Incidenza media', data=country)
c2 = rolling_avg('AnzahlTotTaeglich', title='Nuovi deceduti', data=country)
c3 = rolling_avg('New_Tests', title='Casi testati', data=country)
c4 = rolling_avg('Positivity_Rate', title='Tasso positivita', data=country)
#c6 = rolling_avg('FZICU', title='Terapia intensiva', data=country)
alt.vconcat(c1, c2, c3, c4, c5, c6)

alt.VConcatChart(...)

In [6]:
alt.Chart(df).mark_line().encode(
    x = 'Date:T',
    y = alt.Y('Incidence:Q', title=''),
    row = 'Bundesland_x'
).properties(
    height=100,
    title={"text": "Nuovi casi per 1M di popolazione", "subtitle": ["Dato mediato su 7 giorni"]}
)

alt.Chart(...)